# **Analiza Sezon is Košarkaške Zveze NBA**

Podatki so pobrani iz spletne strani [Basketball Reference](https://www.basketball-reference.com/). Poiskal bom največje število zmag znotraj ene sezone in poiskal najbolj dominantno sezono.

Definiranje konstant in osnovnih funkcij za urejanje podatkov. V funkciji Scraper je dodan `time.sleep(30)` zaradi pogoste napake spletne strani `429 - Too many requests`. Scrapanje traja 15minut :\)

In [1]:
import requests
from bs4 import BeautifulSoup
import time

ALL_TEAMS = ("ATL", "BOS", "NJN", "CHA", "CHI", "CLE", "DAL", "DEN", "DET", "GSW", 
"HOU", "IND", "LAC", "LAL", "MEM", "MIA", "MIL", "MIN", "NOH", "NYK", "OKC", 
"ORL", "PHI", "PHO", "POR", "SAC", "SAS", "TOR", "UTA", "WAS")

def Scraper(teams: tuple) -> dict:
    """Scrape statistics from the desired teams and year dates"""
    data = dict()
    for team in teams:
        time.sleep(30)
        data[team] = dict()
        request = requests.get(f"https://www.basketball-reference.com/teams/{team}/")
        soup = Table_rows(BeautifulSoup(request.text, 'html.parser'))
        for row in soup[1:]:
            data[team][Season_year(row)] = dict()
            for cell in Table_cells(row):
                name = cell["data-stat"]
                if name not in {"DUMMY", "lg_id", "coaches", "top_ws", "rank_team_playoffs", "team_name"}:
                    data[team][Season_year(row)][name] = cell.text
    return data

def Table_rows(soup: str) -> list:
    """Takes HTML5 and returns table rows"""
    return soup.find("table").find_all("tr")

def Table_cells(row):
    """Returns all tables cell from given row"""
    return row.find_all("td")

def Season_year(row):
    """Returns the year of the season"""
    return row.find_all("th")[0].text

data = Scraper(ALL_TEAMS)

Funkcija s katero bom sortiral po zmagah. Napisana je tako, da lahko iščem po katerikoli statistiki.

In [2]:
def Sort_teams(teams: dict[str, dict[str, dict[str, int]]], sort_value: str) -> list[tuple[int, str, str]]:
    scores: list[tuple[int, str, str]] = []
    for team, team_val in teams.items():
        for year, year_val in team_val.items():
            scores.append((int(year_val[sort_value]), year, team))
    return sorted(scores)[::-1]

Spodaj je prikazanih 5 najbolj zmagovitih ekip in sezone v katerih so bile zmage osvojene. Te so Golden State Warriors leta 2016 z 73, Chicago Bulls leta 1996 z 72 in leta 1997 z 69. Los Angeles Lakers leta 1972 z 69 zmagami in Boston Celtics leta 1973 z 68 zmagami.

In [3]:
Sort_teams(data, "wins")[:5]

[(73, '2015-16', 'GSW'),
 (72, '1995-96', 'CHI'),
 (69, '1996-97', 'CHI'),
 (69, '1971-72', 'LAL'),
 (68, '1972-73', 'BOS')]

Naslednja funkcija ločuje ekipe po mojem kriteriju dominance. Sešteje odstotek zmag, število točk v 100ih posegih in število dovoljenih točk v stotih posegih (to število je negativno zato sem ga spremenil v pozitivnega). Zaradi praznih nizov sem ustvaril pomožno funkcijo ki jih spremeni v ničle.

In [4]:
def Float_Zero(value):
    try:
        return float(value)
    except:
        return 0.0

def Sort_by_dominance(teams: dict[str, dict[str, dict[str, int]]]) -> list[tuple[int, str, str]]:
    scores: list[tuple[int, str, str]] = []
    for team, team_val in teams.items():
        for year, year_val in team_val.items():
            scores.append(((10 * Float_Zero(year_val["win_loss_pct"]) + (Float_Zero(year_val["off_rtg_rel"]) / 100) + (Float_Zero(year_val["def_rtg_rel"]) / -100)), year, team))
    return sorted(scores)[::-1]

Opazimo, da so najbolj dominantne ekipe tiste, ki so bile tekom let tudi najbolj hvaljene. Te so Golden State Warriors leta 2017, Chicago bulls v koncu 90ih, Los Angeles Lakers v 70ih in Philadelphia 76ers v 60ih.

In [5]:
Sort_by_dominance(data)[:5]

[(9.007, '2015-16', 'GSW'),
 (8.914, '1995-96', 'CHI'),
 (8.53, '1996-97', 'CHI'),
 (8.515, '1971-72', 'LAL'),
 (8.476, '1966-67', 'PHI')]

Pomožna funkcija, ki prešteje kolikokrat se ponovi določeno število v seznamu.

In [6]:
def Count_occurence(data: list[tuple]) -> dict:
    """Counts how many times a number is listed in given list"""
    count = {}
    for i in data:
        wins = i[0]
        if wins in count:
            count[wins] += 1
        else:
            count[wins] = 1
    return count

Opazimo odstopanje v številu sezon z 0 zmagami. To je posledica letošnje sezone, ki se še ni začela/končala.

In [9]:
count = Count_occurence(Sort_teams(data, "wins"))
count

{73: 1,
 72: 1,
 69: 2,
 68: 2,
 67: 7,
 66: 4,
 65: 5,
 64: 5,
 63: 8,
 62: 16,
 61: 11,
 60: 19,
 59: 21,
 58: 21,
 57: 34,
 56: 29,
 55: 36,
 54: 31,
 53: 28,
 52: 31,
 51: 36,
 50: 46,
 49: 43,
 48: 52,
 47: 55,
 46: 39,
 45: 47,
 44: 64,
 43: 42,
 42: 61,
 41: 64,
 40: 53,
 39: 39,
 38: 40,
 37: 38,
 36: 49,
 35: 33,
 34: 38,
 33: 50,
 32: 27,
 31: 42,
 30: 40,
 29: 32,
 28: 29,
 27: 28,
 26: 26,
 25: 27,
 24: 25,
 23: 23,
 22: 31,
 21: 28,
 20: 19,
 19: 20,
 18: 9,
 17: 19,
 16: 5,
 15: 15,
 14: 4,
 13: 3,
 12: 3,
 11: 2,
 10: 1,
 9: 2,
 8: 1,
 7: 1,
 0: 30}